# APIs Modules - ein erste Blick auf "Requests"

Gezielt und automatisiert Daten im Web auslesen

Zuerst installieren wir pandas und requests. Das können wir von der Command line mit den Befehlen ```pip install requests``` und ```pip install pandas``` die beiden Module installieren. Oder direkt hier in Jupyter Notebook mit ```!pip install requests``` und ```!pip install pandas```.

In [11]:
!pip install requests

In [12]:
!pip install pandas

### Unsere Imports

In [13]:
import requests
import pandas as pd
import calendar, datetime, time

Zur Dokumentation von requestst geht es [hier lang](http://docs.python-requests.org/en/master/).

### Notes

In einer ersten Übung arbeiten wir mit [der API des USGS](https://earthquake.usgs.gov/fdsnws/event/1/#methods) arbeiten. Für die meisten API muss man sich heutezutage eine eigene API-Key kreieren. Das werden wir zu einem späteren Zeitpunkt tun. Diese API braucht keine Keys. Deshalb ist es ein gutes Kennenlernbeispiel.

Falls Probleme mit dem Output auftauchen, hier ein Work around für die Meldung, dass der Output limitiert wurde. [Reconfigure line output](https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-when-viewing-image-in-jupyter-notebook) and [opening stuff from the command line](https://stackoverflow.com/questions/16344709/how-to-open-a-file-from-the-command-line-with-a-specified-program)

### Dokumentation 

Bevor man mit deiner API arbeitet, lohnt es sich immer zuerst, die [Dokumentation](https://earthquake.usgs.gov/fdsnws/event/1/#parameters) anzuschauen. 

### Erste Abfragen

Kleinere Abfrage. Wieviele Erdbeben hat es in einem bestimmten Zeitraum gegeben?#APIs die genutzt werden wollen, dokumentieren ihren Aufbau auf der Website - zB ws-old.parlament

In [14]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20&endtime=2017-09-21'

In [15]:
response = requests.get(url)

In [16]:
response

<Response [200]>

In [17]:
response.text

'312'

Wieviele hat es letzte Nacht von 22 bis 6 Uhr morgens gegeben? Dafür müssen wir in der Dokumentation Time nachfragen.

In [18]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20T22:00:00&endtime=2017-09-21T06:00:00'#Funktion bis ? Dann Parameter mit & verbunden

Wegen Zeitverschiebung Zeit anpassen.

In [19]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/count?starttime=2017-09-20T24:00:00&endtime=2017-09-21T08:00:00'

In [20]:
response = requests.get(url)
response.text

'84'

### Grosse, komibinierte Abfrage

Details zu allen Erbeben in den über drei Tage, 17-9-2017 bis 21-9-2017.

In [21]:
url1 = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'#query heisst wir suchen bestimmte erdbeben

In [22]:
urlzeit = '&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00'

In [23]:
response = requests.get(url1+urlzeit)#Ich hänge die beiden Variablen zusammen
response.text#Alle Erdbeben auf der ganzen Welt in diesem Zeitraum

'{"type":"FeatureCollection","metadata":{"generated":1506884583000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00","title":"USGS Earthquakes","status":200,"api":"1.5.8","count":1035},"features":[{"type":"Feature","properties":{"mag":1.2,"place":"140km WNW of Haines Junction, Canada","time":1505980470681,"updated":1506448261339,"tz":-480,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/ak16874635","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak16874635&format=geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"reviewed","tsunami":0,"sig":22,"net":"ak","code":"16874635","ids":",ak16874635,","sources":",ak,","types":",geoserve,origin,phase-data,","nst":null,"dmin":null,"rms":0.45,"gap":null,"magType":"ml","type":"earthquake","title":"M 1.2 - 140km WNW of Haines Junction, Canada"},"geometry":{"type":"Point","coordinates":[-139.9567,61.1836,3.9]},"id":"ak16874635"},\

Alle Erbeben mit den ungefähren Koordinaten von Mexiko. [Hier könnte man die ungefähr nachschlagen.](http://www.latlong.net/)

In [24]:
urlloc = '&minlatitude=13&maxlatitude=33&minlongitude=-120&maxlongitude=-85'

In [25]:
response = requests.get(url1+urlzeit+urlloc)
response.text

'{"type":"FeatureCollection","metadata":{"generated":1506884584000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00&minlatitude=13&maxlatitude=33&minlongitude=-120&maxlongitude=-85","title":"USGS Earthquakes","status":200,"api":"1.5.8","count":34},"features":[{"type":"Feature","properties":{"mag":2.26,"place":"4km SW of Imperial, CA","time":1505947525530,"updated":1505948918700,"tz":-480,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"reviewed","tsunami":0,"sig":79,"net":"ci","code":"38007144","ids":",ci38007144,","sources":",ci,","types":",focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,","nst":41,"dmin":0.08931,"rms":0.26,"gap":102,"magType":"ml","type":"earthquake","title":"M 2.3 - 4km SW of Imperial, CA"},"geomet

Dieser Wulst lässt sich aber ziemlich schwer lesen. Deshalb müssen wir das zuerst umformatieren und dann verstehen, wie das ganze strukturiert ist. 

In [26]:
response.json()#json strukturiert die daten. Ich sehe es ist ein dictionyry von zwei listen, die wiederum dictionaries enthält mit keys  und values - wo beginnt sie; es nur zwei keys: bbox und features. features ist interessanter

{'bbox': [-117.0765, 14.5296, -0.2, -90.9069, 32.9196667, 177.24],
 'features': [{'geometry': {'coordinates': [-115.6006667, 32.8185, 11.77],
    'type': 'Point'},
   'id': 'ci38007144',
   'properties': {'alert': None,
    'cdi': None,
    'code': '38007144',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson',
    'dmin': 0.08931,
    'felt': None,
    'gap': 102,
    'ids': ',ci38007144,',
    'mag': 2.26,
    'magType': 'ml',
    'mmi': None,
    'net': 'ci',
    'nst': 41,
    'place': '4km SW of Imperial, CA',
    'rms': 0.26,
    'sig': 79,
    'sources': ',ci,',
    'status': 'reviewed',
    'time': 1505947525530,
    'title': 'M 2.3 - 4km SW of Imperial, CA',
    'tsunami': 0,
    'type': 'earthquake',
    'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,',
    'tz': -480,
    'updated': 1505948918700,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144'},
   'type': 'Feature'},

In [27]:
dct = response.json()

Um sicher zu gehen, prüfen wir nochmals den Datentyp.

In [28]:
type(dct)

dict

Schauen wir uns alle keys und values an

In [29]:
for key, value in dct.items() :# mit .items angehängt an dictionnary-name druckt value und keys aus
    print (key, value)

type FeatureCollection
metadata {'generated': 1506884584000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2017-09-17T24:00:00&endtime=2017-09-21T08:00:00&minlatitude=13&maxlatitude=33&minlongitude=-120&maxlongitude=-85', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.5.8', 'count': 34}
features [{'type': 'Feature', 'properties': {'mag': 2.26, 'place': '4km SW of Imperial, CA', 'time': 1505947525530, 'updated': 1505948918700, 'tz': -480, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson', 'felt': None, 'cdi': None, 'mmi': None, 'alert': None, 'status': 'reviewed', 'tsunami': 0, 'sig': 79, 'net': 'ci', 'code': '38007144', 'ids': ',ci38007144,', 'sources': ',ci,', 'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,', 'nst': 41, 'dmin': 0.08931, 'rms': 0.26, 'gap': 102, 'magType': 'ml', 'type': 'earthqua

Features interessiert uns.

In [30]:
type(dct['features'][0])#type zeigt mir den Datentyp; mit (0) sage ich, dass er mir erstes der Features

dict

In [31]:
len(dct['features'])

34

Schauen wir uns das erste Element an

In [32]:
for key in dct['features'][0]:#so frage ich die Keys im Value Feature ab
    print(key)

type
properties
geometry
id


In [33]:
dct['features'][0]#so frage ich das erste element von features ab

{'geometry': {'coordinates': [-115.6006667, 32.8185, 11.77], 'type': 'Point'},
 'id': 'ci38007144',
 'properties': {'alert': None,
  'cdi': None,
  'code': '38007144',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci38007144&format=geojson',
  'dmin': 0.08931,
  'felt': None,
  'gap': 102,
  'ids': ',ci38007144,',
  'mag': 2.26,
  'magType': 'ml',
  'mmi': None,
  'net': 'ci',
  'nst': 41,
  'place': '4km SW of Imperial, CA',
  'rms': 0.26,
  'sig': 79,
  'sources': ',ci,',
  'status': 'reviewed',
  'time': 1505947525530,
  'title': 'M 2.3 - 4km SW of Imperial, CA',
  'tsunami': 0,
  'type': 'earthquake',
  'types': ',focal-mechanism,geoserve,nearby-cities,origin,phase-data,scitech-link,',
  'tz': -480,
  'updated': 1505948918700,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci38007144'},
 'type': 'Feature'}

Lesen wir folgendes aus:
1. Type
2. Magnitude
4. Örtlichkeit
5. Time 

In [35]:
earthquakes = []#Leere Liste kreieren

for elem in dct['features']:
    t = elem['properties']['type']#in den properties (die ich ja vorhin als element gefunden habe) will ich 
    m = elem['properties']['mag']
    p = elem['properties']['place']
    tm = elem['properties']['time']
    
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}#Hier baue ich mein eigenes Dictionary auf; die Begriffe kann ich selbst wählen
    
    earthquakes.append(mini_dict)

Dealing with Unix time

In [92]:
earthquakes

[{'Mag': 2.26,
  'Place': '4km SW of Imperial, CA',
  'Time': 1505947525530,
  'Type': 'earthquake'},
 {'Mag': 1,
  'Place': '7km NW of La Mesa, CA',
  'Time': 1505937649230,
  'Type': 'quarry blast'},
 {'Mag': 2.71,
  'Place': '19km W of Delta, B.C., MX',
  'Time': 1505931747440,
  'Type': 'earthquake'},
 {'Mag': 0.94,
  'Place': '27km SSW of Ocotillo Wells, CA',
  'Time': 1505922423230,
  'Type': 'earthquake'},
 {'Mag': 2.21,
  'Place': '7km N of Delta, B.C., MX',
  'Time': 1505911740560,
  'Type': 'earthquake'},
 {'Mag': 4.7,
  'Place': '51km SSW of Pijijiapan, Mexico',
  'Time': 1505903968940,
  'Type': 'earthquake'},
 {'Mag': 4.6,
  'Place': '15km ESE of San Mateo del Mar, Mexico',
  'Time': 1505878085370,
  'Type': 'earthquake'},
 {'Mag': 4.9,
  'Place': '4km ESE of Acatenango, Guatemala',
  'Time': 1505875055380,
  'Type': 'earthquake'},
 {'Mag': 4.6,
  'Place': '15km SW of San Dionisio del Mar, Mexico',
  'Time': 1505862694420,
  'Type': 'earthquake'},
 {'Mag': 4.9,
  'Place': 

In [36]:
earthquakes = []

for elem in dct['features']:
    t = elem['properties']['type']
    m = elem['properties']['mag']
    p = elem['properties']['place']
    
    #Ints bearbeiten, die letzten drei Stellen abschneiden (weil der Converter von Unixtime in Normalzeit die Angabe nur lesen kann, wenn die letzten drei Zahlen weg sind); ich muss zahl zuerst in string verwandeln, weil man nur dort die drei letzten zeichen abschneiden kann; dann verwandle ich es wieder in eine zahl zurück und mit dt
    tm = dt(int(str(elem['properties']['time'])[:-3])) 
    
    mini_dict = {'Type': t,
                 'Mag': m,
                 'Place': p,
                 'Time': tm}
    
    earthquakes.append(mini_dict)

NameError: name 'dt' is not defined

In [ ]:
earthquakes

### Erste Erfahrung mit Pandas

In [97]:
pd.DataFrame(earthquakes)#es muss aber eine dictionary-Liste sein; kann auch nur eine Liste sein, dann gibt es nur eine Spalte

Empty DataFrame
Columns: []
Index: []

In [38]:
df = pd.DataFrame(earthquakes)

In [39]:
df.sort_values(by='Mag', ascending=False)#startet von 'Mag'; default Einstellung: beginnt beim kleinsten, wenn ich beim stärksten beginnen will, muss ich ascending=false eingeben

KeyError: 'Mag'

Speichern wir das ab.

In [67]:
df.to_csv('erdbeben.csv')

NameError: name 'df' is not defined